<a href="https://colab.research.google.com/github/Allay-Sizo/Day4/blob/main/tut4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import apache_beam as beam


def format_record(record):
    return f"{record['id']};{record['name']};{record['gender']};{record['age']}; " \
           f"{record['city']},{record['state']},{record['zip']};{record['date']}"


data = [
    {"id": 1, "name": "Amy Sullivan", "gender": "Female", "age": 20,
     "city": "Westlake", "state": "OH", "zip": "44145", "date": "2020-08-31"},
    {"id": 1, "name": "Paige Dixon", "gender": "Female", "age": 43,
     "city": "Hicksville", "state": "NY", "zip": "11801", "date": "2020-03-22"}
]

with beam.Pipeline() as pipeline:
    (
        pipeline
        | "Create input" >> beam.Create(data)
        | "Format records" >> beam.Map(format_record)
        | "Print output" >> beam.Map(print)
    )


1;Amy Sullivan;Female;20; Westlake,OH,44145;2020-08-31
1;Paige Dixon;Female;43; Hicksville,NY,11801;2020-03-22


In [ ]:
import apache_beam as beam
from apache_beam.options.pipeline_options import PipelineOptions


pipeline_options = PipelineOptions()
with beam.Pipeline(options=pipeline_options) as pipeline:

    (
        pipeline
        | "Create sample data" >> beam.Create([
            {"id": 1, "name": "Amy Sullivan", "gender": "Female", "age": 20,
             "city": "Westlake", "state": "OH", "zip": "44145", "date": "2020-08-31"},
            {"id": 2, "name": "Paige Dixon", "gender": "Female", "age": 43,
             "city": "Hicksville", "state": "NY", "zip": "11801", "date": "2020-03-22"},
            {"id": 3, "name": "John Smith", "gender": "Male", "age": 35,
             "city": "Cleveland", "state": "OH", "zip": "44109", "date": "2020-05-11"},
            {"id": 4, "name": "Maria Lopez", "gender": "Female", "age": 29,
             "city": "Miami", "state": "FL", "zip": "33101", "date": "2020-06-15"},
            {"id": 5, "name": "James Carter", "gender": "Male", "age": 52,
             "city": "Dallas", "state": "TX", "zip": "75201", "date": "2020-07-20"},
            {"id": 6, "name": "Sophia Chen", "gender": "Female", "age": 31,
             "city": "Seattle", "state": "WA", "zip": "98101", "date": "2020-02-19"},
            {"id": 7, "name": "Michael Brown", "gender": "Male", "age": 27,
             "city": "Denver", "state": "CO", "zip": "80202", "date": "2020-04-05"},
            {"id": 8, "name": "Emily Davis", "gender": "Female", "age": 38,
             "city": "Boston", "state": "MA", "zip": "02108", "date": "2020-09-12"}
        ])
        | "Map gender to count" >> beam.Map(lambda r: (r['gender'], 1))
        | "Count per gender" >> beam.CombinePerKey(sum)
        | "Format output" >> beam.Map(lambda kv: f"{kv[0]}: {kv[1]}")
        | "Print results" >> beam.Map(print)
    )


Female: 5
Male: 3


In [ ]:
import apache_beam as beam
from apache_beam.options.pipeline_options import PipelineOptions

def to_kv_state(_row):
    return _row['state'], 1

def fmt(kv):
    state, count = kv
    return f"{state}: {count}"

if __name__ == "__main__":

    pipeline_options = PipelineOptions()
    with beam.Pipeline(options=pipeline_options) as pipeline:

        counts = (
            pipeline
            | "Create sample data" >> beam.Create([
                {"id": 1, "name": "Amy Sullivan", "gender": "Female", "age": 20,
                 "city": "Westlake", "state": "OH", "zip": "44145", "date": "2020-08-31"},
                {"id": 2, "name": "Paige Dixon", "gender": "Female", "age": 43,
                 "city": "Hicksville", "state": "NY", "zip": "11801", "date": "2020-03-22"},
                {"id": 3, "name": "John Smith", "gender": "Male", "age": 35,
                 "city": "Cleveland", "state": "OH", "zip": "44109", "date": "2020-05-11"},
                {"id": 4, "name": "Maria Lopez", "gender": "Female", "age": 29,
                 "city": "Miami", "state": "FL", "zip": "33101", "date": "2020-06-15"},
                {"id": 5, "name": "James Carter", "gender": "Male", "age": 52,
                 "city": "Dallas", "state": "TX", "zip": "75201", "date": "2020-07-20"},
                {"id": 6, "name": "Sophia Chen", "gender": "Female", "age": 31,
                 "city": "Seattle", "state": "WA", "zip": "98101", "date": "2020-02-19"},
                {"id": 7, "name": "Michael Brown", "gender": "Male", "age": 27,
                 "city": "Denver", "state": "CO", "zip": "80202", "date": "2020-04-05"},
                {"id": 8, "name": "Emily Davis", "gender": "Female", "age": 38,
                 "city": "Boston", "state": "MA", "zip": "02108", "date": "2020-09-12"}
            ])
            | "Map state to 1" >> beam.Map(to_kv_state)
            | "Count per state" >> beam.CombinePerKey(sum)
        )


        _ = counts | "Format for print" >> beam.Map(fmt) | "Print" >> beam.Map(print)


        _ = (counts
             | "Format for file" >> beam.Map(fmt)
             | "Write to text" >> beam.io.WriteToText("state_counts", num_shards=1))


OH: 2
NY: 1
FL: 1
TX: 1
WA: 1
CO: 1
MA: 1


Watermarks – Like deciding the day’s sales are done even if a few delayed transactions from the card machine might still arrive.


Triggers – Sending interim sales reports during the day instead of waiting until the store closes.


Allowed lateness – Choosing how long you’ll still accept yesterday’s late sales data before finalizing the daily totals.